In [1]:
import pandas
import numpy as np
import os
import glob
import plotly.express as px
import plotly.subplots as sp
import statsmodels

from load import *

pandas.set_option('display.max_columns', 200)
pandas.set_option('display.max_rows', 80)

In [2]:
btp_raw = pandas.concat([load_csv(path) for path in glob.glob("../results/ns-3/*_*_true_*_*_*_*_*.csv")])

In [3]:
sbp_raw = pandas.concat([load_data(path) for path in glob.glob("../results/ns-3/*_*_false_*_*_*_*_*.log")])

In [4]:
raw = pandas.concat([btp_raw, sbp_raw])
raw["Construction Time (s)"] = raw["Time to Build (ms)"] / 1000

In [5]:
raw.reset_index(inplace=True)

In [6]:
cycle_color_map = {
    "Mutex": "#636EFA",
    "Path to Source": "#EF553B",
    "Ping to Source": "#00CC96",
}

## Filter & Present Invalid Runs

In [7]:
invalid = raw[
    (raw["Unconnected Nodes"] > 1)
    | (raw["Cycles Lasted"] > 0.0)
]

In [8]:
df = raw.loc[
    (raw["Unconnected Nodes"] <= 1)
    & (raw['Cycles Lasted'] == 0)
]

In [20]:
means = invalid.groupby(["Node Count", "Cycle Prevention", "Width"]).count().sort_values("Width")
means
fig = px.bar(
    means, 
    x=means.index.get_level_values(0).astype(str), 
    color=means.index.get_level_values(1),
    color_discrete_map=cycle_color_map,
    y="Run",
    barmode='group',
    facet_col=means.index.get_level_values(2),
    labels={
        "x": "Number of Nodes",
        "y": "Unfinished Experiments",
        "color": "Cycle Handling",
        "facet_col": "Area Conf.",
    },
    category_orders={
        "color": ["Mutex", "Path to Source", "Ping to Source"]
    }
)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ' ')))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("100", "(1)")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("200", "(2)")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("300", "(3)")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("400", "(4)")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("500", "(5)")))

fig.update_xaxes(categoryorder='array', categoryarray= ['50','100','150','200', '250', '300'])
fig.update_xaxes(tickangle=90)

area_labels = {
    'Path to Source':'Path-to-Source',
    'Mutex':'Mutex',
    'Ping to Source':'Ping-to-Source',
}
fig.for_each_trace(lambda t: t.update(name = area_labels[t.name], legendgroup = area_labels[t.name], hovertemplate = t.hovertemplate.replace(t.name, area_labels[t.name])))


fig.write_image("unfinished_experiments.pdf", width=1000, height=300)
fig.show()

In [21]:
counts = raw[(raw['Cycles Lasted'] > 0) & (raw['Cycle Prevention'] == "Path to Source")].groupby(["Node Count", "Cycle Prevention", "Width"]).count().sort_values("Width")

fig = px.bar(
    counts, 
    x=counts.index.get_level_values(0), 
    y="Cycles Lasted",
    color=counts.index.get_level_values(2).astype(str),
    barmode='group',    
    labels={
        "x": "Number of Nodes",
        "y": "Unresolved Cycles",
        "color": "Area Conf.",
    },
)

area_labels = {
    '100':'(1)',
    '200':'(2)',
    '300':'(3)',
    '400':'(4)',
    '500':'(5)',
}

fig.for_each_trace(lambda t: t.update(name = area_labels[t.name], legendgroup = area_labels[t.name], hovertemplate = t.hovertemplate.replace(t.name, area_labels[t.name])))

fig.write_image("path_to_source_cycles.pdf", width=500, height=300)
fig.show()

Number of experiements where at least one cycle could not be resolved.

In [22]:
means = raw.groupby(["Node Count", "Cycle Prevention", "Width"]).mean()

means['Unconnected Nodes (%)'] = means['Unconnected Nodes'] / means.index.get_level_values(0) * 100

fig = px.line(
    means, 
    x=means.index.get_level_values(0), 
    y="Unconnected Nodes (%)", 
    color=means.index.get_level_values(2), 
    labels={
        "x": "Number of Nodes",
        "color": "Area Conf.",
        "facet_col": "Cycle Handling",
    },
    facet_col=means.index.get_level_values(1),
)

area_labels = {
    '100':'(1)',
    '200':'(2)',
    '300':'(3)',
    '400':'(4)',
    '500':'(5)',
}

fig.for_each_trace(lambda t: t.update(name = area_labels[t.name], legendgroup = area_labels[t.name], hovertemplate = t.hovertemplate.replace(t.name, area_labels[t.name])))

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Mutex", "Mutex")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Path to Source", "Path-to-Source")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Ping to Source", "Ping-to-Source")))

fig.update_xaxes(categoryorder='array', categoryarray= ['50','100','150','200', '250', '300'])
fig.update_xaxes(tickangle=90)

fig.write_image("unconnected_nodes.pdf", width=1000, height=300)
fig

Percentage of unconnected nodes for differenet CPMs, node counts and area sizes.

## Create plots: Performance Analysis

In [23]:
means = df[df["BTP"] == True].groupby(["Node Count", "Cycle Prevention", "Width"]).mean()

fig = px.line(
    means, 
    x=means.index.get_level_values(0).astype(str), 
    y='Construction Time (s)',
    color=means.index.get_level_values(1),
    color_discrete_map=cycle_color_map,
    labels={
        "x": "Number of Nodes",
        "color": "Cycle Handling",
        "facet_col": "Area Conf.",
    },
    facet_col=means.index.get_level_values(2),
    category_orders={
        "color": ["Mutex", "Path to Source", "Ping to Source"]
    }
)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ' ')))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("100", "(1)")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("200", "(2)")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("300", "(3)")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("400", "(4)")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("500", "(5)")))

fig.update_xaxes(categoryorder='array', categoryarray= ['50','100','150','200', '250', '300'])
fig.update_xaxes(tickangle=90)

area_labels = {
    'Path to Source':'Path-to-Source',
    'Mutex':'Mutex',
    'Ping to Source':'Ping-to-Source',
}
fig.for_each_trace(lambda t: t.update(name = area_labels[t.name], legendgroup = area_labels[t.name], hovertemplate = t.hovertemplate.replace(t.name, area_labels[t.name])))

fig.write_image("time_to_build.pdf", width=1000, height=300)
fig

Time to Build for different cycle prevention methods, number of nodes and area sizes.

In [24]:
means = df[df["BTP"] == True].groupby(["Node Count", "Cycle Prevention", "Width"]).mean()

fig = px.line(
    means, 
    x=means.index.get_level_values(0).astype(str), 
    y="Total TX Power", 
    color=means.index.get_level_values(2), 
    labels={
        "x": "Number of Nodes",
        "Total TX Power": "Cumulative Transmission Power (W)",
        "color": "Area Conf.",
        "facet_col": "Cycle Handling",
    },
    facet_col=means.index.get_level_values(1),
)

area_labels = {
    '100':'(1)',
    '200':'(2)',
    '300':'(3)',
    '400':'(4)',
    '500':'(5)',
}

fig.for_each_trace(lambda t: t.update(name = area_labels[t.name], legendgroup = area_labels[t.name], hovertemplate = t.hovertemplate.replace(t.name, area_labels[t.name])))

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Mutex", "Mutex")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Path to Source", "Path-to-Source")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Ping to Source", "Ping-to-Source")))

fig.update_xaxes(categoryorder='array', categoryarray= ['50','100','150','200', '250', '300'])
fig.update_xaxes(tickangle=90)

fig.write_image("cumulative_tx_power.pdf", width=1000, height=300)
fig

Cumulative transmission power for different CPMs, number of nodes and area sizes.

In [25]:
groups = ["Energy Overhead", "Energy ApplicationData"]
id_vars = [col for col in df if col not in groups]

melted = pandas.melt(df[df["BTP"] == True], id_vars=id_vars, var_name="Energy Type", value_name="Energy")
melted["Energy Type"].replace("Energy Overhead", "Construction", inplace=True)
melted["Energy Type"].replace("Energy ApplicationData", "Data", inplace=True)

means = melted.groupby(["Node Count", "Cycle Prevention", "Width", "Energy Type"]).mean()

fig = px.line(
    means, 
    x=means.index.get_level_values(0).astype(str), 
    y=means["Energy"],
    facet_col=means.index.get_level_values(1),
    color=means.index.get_level_values(2), 
    line_dash=means.index.get_level_values(3), 
    line_dash_sequence=["4px", "solid"],
    labels={
        "x": "Number of Nodes",
        "Energy": "Energy (J)",
        "color": "Area Conf.",
        "line_dash": "Category",
    },
)

area_labels = {
    '100, Construction':'(1), Construction',
    '100, Data':'(1), Data',
    '200, Construction':'(2), Construction',
    '200, Data':'(2), Data',
    '300, Construction':'(3), Construction',
    '300, Data':'(3), Data',
    '400, Construction':'(4), Construction',
    '400, Data':'(4), Data',
    '500, Construction':'(5), Construction',
    '500, Data':'(5), Data',
}

fig.for_each_trace(lambda t: t.update(name = area_labels[t.name], legendgroup = area_labels[t.name], hovertemplate = t.hovertemplate.replace(t.name, area_labels[t.name])))

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Mutex", "Mutex")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Path to Source", "Path-to-Source")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Ping to Source", "Ping-to-Source")))

fig.update_xaxes(categoryorder='array', categoryarray= ['50','100','150','200', '250', '300'])
fig.update_xaxes(tickangle=90)

fig.write_image("energy_overhead.pdf", width=1000, height=300)
fig

In [26]:
means = df[df["BTP"] == True].groupby(["Node Count", "Cycle Prevention", "Width"]).mean()

fig = px.line(
    means, 
    x=means.index.get_level_values(0), 
    y="Tree Depth", 
    color=means.index.get_level_values(2), 
    labels={
        "x": "Number of Nodes",
        "color": "Area Conf.",
        "facet_col": "CPM",
    },
    facet_col=means.index.get_level_values(1),
)

area_labels = {
    '100':'(1)',
    '200':'(2)',
    '300':'(3)',
    '400':'(4)',
    '500':'(5)',
}

fig.for_each_trace(lambda t: t.update(name = area_labels[t.name], legendgroup = area_labels[t.name], hovertemplate = t.hovertemplate.replace(t.name, area_labels[t.name])))

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Mutex", "Mutex")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Path to Source", "Path-to-Source")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Ping to Source", "Ping-to-Source")))

fig

In [27]:
total_energy_df = raw[raw["Node Count"] >= 50].sort_values("Node Count")
fig = px.box(
    total_energy_df, 
    facet_col="BTP", 
    y=total_energy_df["Total Energy"], 
    color="Node Count",
    x="Width",
    labels={
        "Width": "Area Conf.",
        "Total Energy": "Total Energy (J)",
        "Node Count": "Number of Nodes",
    },
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("True", 'BTP')))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("False", 'SBP')))

fig.update_layout(
    xaxis1 = dict(
        tickmode = 'array',
        tickvals = [100, 200, 300, 400, 500],
        ticktext = ["(1)", "(2)", "(3)", "(4)", "(5)"]
    ),
    xaxis2 = dict(
        tickmode = 'array',
        tickvals = [100, 200, 300, 400, 500],
        ticktext = ["(1)", "(2)", "(3)", "(4)", "(5)"]
    )
)

fig.write_image("energy_comparison_sbp.pdf", width=1000, height=300)
fig

In [28]:
merged = pandas.merge(
    btp_raw, 
    sbp_raw, 
    left_on=["Node Count", "Width", "Seed", "Cycle Prevention"], 
    right_on=["Node Count", "Width", "Seed", "Cycle Prevention"],
    suffixes=[" BTP", " SBP"],
)
merged["Energy Proportion"] = merged["Total Energy BTP"] / merged["Total Energy SBP"]
merged["Energy Savings (%)"] = (merged["Energy Proportion"]) * 100

fig = px.box(
    merged.sort_values("Node Count"), 
    x="Width",
    y="Energy Savings (%)", 
    color="Node Count",    
    labels={
        "Width": "Area Conf.",
        "Node Count": "Nuber of Nodes",
        "Energy Savings (%)": "Energy compared to SBP (%)",
        "100": "(1)"
    },
)

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [100, 200, 300, 400, 500],
        ticktext = ["(1)", "(2)", "(3)", "(4)", "(5)"]
    )
)

fig.write_image("energy_savings.pdf", width=700, height=400)
fig